In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='g34b0jvjD6UXE1mHfi5cnsYA53qMHDzbU1u0moqvMqrF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'arrythmiaclassification-donotdelete-pr-5zcb8m4jmpsq73'
object_key = 'data.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
pwd

'/home/wsuser/work'

## IMAGE PROCESSING

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip = True, vertical_flip = True, zoom_range = 0.2)

In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
x_train = train_datagen.flow_from_directory("/home/wsuser/work/data/train", target_size = (64,64), batch_size = 100, 
                                            class_mode = "categorical")

Found 15341 images belonging to 6 classes.


In [9]:
x_test = test_datagen.flow_from_directory("/home/wsuser/work/data/test", target_size = (64,64), batch_size = 100, 
                                          class_mode = "categorical")

Found 6825 images belonging to 6 classes.


In [8]:
x_train.class_indices

{'Left Bundle Branch Block': 0,
 'Normal': 1,
 'Premature Atrial Contraction': 2,
 'Premature Ventricular Contractions': 3,
 'Right Bundle Branch Block': 4,
 'Ventricular Fibrillation': 5}

## MODEL BUILDING

### Import the Library

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

### Initializing the Model

In [11]:
model = Sequential()

### Adding CNN Layers

In [12]:
model.add(Convolution2D(32, (3,3), input_shape = (64,64,3), activation = "relu"))

In [13]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [14]:
model.add(Convolution2D(32, (3,3), activation = "relu"))

In [15]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [16]:
model.add(Flatten())

### Adding Dense Layers

In [17]:
model.add(Dense(units=128, kernel_initializer='random_uniform', activation="relu"))

In [18]:
model.add(Dense(units=128, kernel_initializer='random_uniform', activation="relu"))

In [19]:
model.add(Dense(units=128, kernel_initializer='random_uniform', activation="relu"))

In [20]:
model.add(Dense(units=128, kernel_initializer='random_uniform', activation="relu"))

In [21]:
model.add(Dense(units=128, kernel_initializer='random_uniform', activation="relu"))

In [23]:
model.add(Dense(units=6, kernel_initializer='random_uniform', activation="softmax"))

### Configure the Learning Process

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [25]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

### Train the Model

In [27]:
model.fit(x_train, steps_per_epoch = len(x_train), epochs=9, validation_data=x_test,\
                    validation_steps = len(x_test))

Epoch 1/9
154/154 [==============================] - 82s 531ms/step - loss: 0.8560 - accuracy: 0.6836 - val_loss: 1.3039 - val_accuracy: 0.5307
Epoch 2/9
154/154 [==============================] - 85s 552ms/step - loss: 0.6373 - accuracy: 0.7776 - val_loss: 0.9838 - val_accuracy: 0.6734
Epoch 3/9
154/154 [==============================] - 85s 550ms/step - loss: 0.4525 - accuracy: 0.8481 - val_loss: 0.8842 - val_accuracy: 0.7228
Epoch 4/9
154/154 [==============================] - 85s 553ms/step - loss: 0.3521 - accuracy: 0.8842 - val_loss: 0.6524 - val_accuracy: 0.7965
Epoch 5/9
154/154 [==============================] - 86s 559ms/step - loss: 0.2478 - accuracy: 0.9216 - val_loss: 0.4875 - val_accuracy: 0.8482
Epoch 6/9
154/154 [==============================] - 89s 576ms/step - loss: 0.1901 - accuracy: 0.9417 - val_loss: 0.5241 - val_accuracy: 0.8469
Epoch 7/9
154/154 [==============================] - 85s 554ms/step - loss: 0.1555 - accuracy: 0.9519 - val_loss: 0.4762 - val_accuracy:

### Save the Model

In [28]:
model.save('ECG.h5')

In [29]:
!tar -zcvf ECG-arrhythmia-classification-model_new.tgz ECG.h5

ECG.h5


In [30]:
ls -1

data/
ECG-arrhythmia-classification-model_new.tgz
ECG.h5


In [31]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 22.1 MB/s eta 0:00:01


In [33]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"XKr8dWsa1JPjnDh8Xf5E--Uv3Ef_vdN2deYyEdj2rEth"
}
client=APIClient(wml_credentials)

In [34]:
client=APIClient(wml_credentials)

In [36]:
def guid_from_space_name(client, space_name):
    space = client_spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [37]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------  ------------------------
ID                                    NAME            CREATED
1a6dcfa8-a632-4da1-b2e3-1e28f2cc97a3  Classification  2022-11-15T13:02:02.173Z
------------------------------------  --------------  ------------------------


In [38]:
space_uid='1a6dcfa8-a632-4da1-b2e3-1e28f2cc97a3'

In [39]:
client.set.default_space(space_uid)

'SUCCESS'

In [40]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [41]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [42]:
model_details = client.repository.store_model(model='ECG-arrhythmia-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN Model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [43]:
model_id

'0f611e10-dcf0-4a70-b8c8-6c5269bde33c'

In [44]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

### Test the Model

In [45]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [49]:


bucket = 'arrythmiaclassification-donotdelete-pr-5zcb8m4jmpsq73'
object_key = 'Unknown_image.png'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [59]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model=load_model('/home/wsuser/work/ECG.h5')
img=image.load_img('data/test/Left Bundle Branch Block/fig_5897.png', target_size=(64, 64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = model.predict(x)
y_pred=np.argmax(pred)
y_pred

0

In [60]:
index=['Left Bundle Branch Block',
       'Normal',
       'Premature Atrial Contraction',
       'Premature Ventricular Contractions',
       'Right Bundle Branch Block',
       'Ventricular Fibrillation']

result = str(index[y_pred])
result

'Left Bundle Branch Block'